In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\Ashutosh Mukherjee\AppData\Local\Temp\ipykernel_14452\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


***

# FBS using python: a simple multi-DoF case

> Francesco Trainotti $\,\,$ *francesco.trainotti@tum.de* <br>

> Advanced Structural Dynamics, DTU Copenhagen, 16-19 June 2025





***

## Introduction

The dynamic properties of the system are described by the force balance, where the external forces are equal to the internal forces resulting from inertia, damping, and elasticity. 
The basic equation in the time domain for a linear dynamic system takes the following form:

$$
\mathbf{M} \, \boldsymbol{\ddot{x}}(t) + \mathbf{C} \, \boldsymbol{\dot{x}}(t) + \mathbf{K} \, \boldsymbol{x}(t) = \boldsymbol{f}(t),
$$

where $\mathbf{M}$ represents the mass matrix, $\mathbf{C}$ the damping matrix, $\mathbf{K}$ the stiffness matrix, $\boldsymbol{x}$ vector of responses at all $n$ degrees of freedom (DoFs), and $\boldsymbol{f}$ vector of externally applied forces.

We assume the following properties:

* __Linearity__ - response amplitude is linearly proportional to the excitation amplitude.

* __Time invariance__ - mass, stiffness, and damping characteristics are time-independent.

If we apply the Fourier transform, a time function of response $\boldsymbol{x}(t)$ can be written in  the frequency domain ($\boldsymbol{x}(\omega)$) and the basic equation takes the following form: 

$$
(- \omega^2\,\mathbf{M} + i\omega\,\mathbf{C}+  \mathbf{K}) \boldsymbol{x}(\omega) = \boldsymbol{f}(\omega).
$$

The left-hand side can be collected into a single frequency-dependent matrix $\mathbf{Z}(\omega)$:

$$
\mathbf{Z}(\omega) \boldsymbol{x}(\omega) = \boldsymbol{f}(\omega).
$$

The **impedance matrix** $\mathbf{Z}(\omega)$ is also called the dynamic stiffness matrix. By inverting the mechanical impedance an admittance notation $\mathbf{Y}(\omega)$ can be obtained:

$$
\boldsymbol{u}(\omega) = \mathbf{Y}(\omega) \boldsymbol{f}(\omega). \qquad \mathbf{Y}(\omega) = \left(\mathbf{Z}(\omega)\right)^{-1}.
$$

The $\mathbf{Y}(\omega)$ denotes the frequency response function matrix.

## Simple discrete mass-spring system in Python

Let's first import some packages, which we will use for the calculations.

In [3]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt

Consider a simple discrete dynamic system with 4 degrees of freedom:

<p>
<img src="./pics/A.svg"/ width = 400>
</p>

Now let's define the mass matrix:

$
\textbf{M} = \begin{bmatrix}
m_1 & 0 & 0 & 0 \\ 
0 & m_2 & 0 & 0 \\ 
0 & 0 & m_3 & 0 \\ 
0 & 0 & 0 & m_4 \\ 
\end{bmatrix}
.$


In [4]:
m1 = 1
m2 = 1
m3 = 1
m4 = 1

M = np.asarray([[m1,0,0,0],
                [0,m2,0,0],
                [0,0,m3,0],
                [0,0,0,m4]])
display(M)

array([[1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1]])

Now let's define the stiffness matrix:

$
\textbf{K} = \begin{bmatrix}
k_1 & -k_1 & 0 & 0 \\ 
-k_1 & k_1+k_2 & -k_2 & 0 \\ 
0 & -k_2 & k_2+k_3  & - k_3 \\ 
0 & 0 & -k_3 & k_3 \\ 
\end{bmatrix}
.$


In [5]:
k1 = 200
k2 = 200
k3 = 200

K = np.asarray([[k1,-k1,0,0],
                [-k1,k1+k2,-k2,0],
                [0,-k2,k2+k3,-k3],
                [0,0,-k3,k3]])

display(K)

array([[ 200, -200,    0,    0],
       [-200,  400, -200,    0],
       [   0, -200,  400, -200],
       [   0,    0, -200,  200]])

### Solving eigenvalue problem (of the conservative system)

The eigenfrequencies and eigenvectors  can be determined from the equilibrium equation, when considering a system response with free vibrations:

$$
(\mathbf{K} - \omega^2\,\mathbf{M}) \boldsymbol{X} = \boldsymbol{0}.
$$

Make sure to estimate eigenfrequencies $\pmb{\omega}_{r}$ and mass-normalized eigenvectors $\boldsymbol{\phi}_r$, such that:

$$
\pmb{\Phi}^{\text{T}}\,\mathbf{M}\,\pmb{\Phi} = [\mathbf{I}], 
   \qquad
   \pmb{\Phi}^{\text{T}}\,\mathbf{K}\,\pmb{\Phi} = [^{\nwarrow}{\pmb{\omega}_{r}^2}_{\searrow}].
$$

In [10]:
# use a python eigensolver of choice to get as output (eig_val, eig_vec). Hint: use a dense generalized eigensolver
from scipy import linalg
# M -> right hand matrix, K -> matrix whose eigenvecs are computed
[eig_val, eig_vec] = linalg.eigh(K,M)


eig_val.sort()
eig_freq = np.sqrt(np.abs(np.real(eig_val)))  

print(eig_freq/(2*np.pi))

for i in range(eig_vec.shape[1]):
    phi = eig_vec[:, i]
    mass_norm = phi.T @ M @ phi
    print(mass_norm)

[3.31137421e-08 1.72268069e+00 3.18309886e+00 4.15891909e+00]
0.9999999999999999
1.0000000000000004
0.9999999999999988
0.9999999999999989


In [8]:
M

array([[1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1]])

### Adding damping

Let's add some damping in the form of proportional/Rayleigh, i.e.
$
\mathbf{C}=\alpha \mathbf{M} + \beta \mathbf{K}
$

This translates in the following modal relationship:
$
\zeta_i \;=\; \frac{1}{2}(\frac{\alpha}{\omega_i} \;+\; \beta\,\omega_i)
$

Let's compute the rayleigh coefficients given modal damping 0.2\% and 0.3\% at first and third flexible mode

In [ ]:
# build a function rayleigh_coeffs that takes (omegas, zetas) as inputs and get (alpha, beta) as outputs. Hint: use np.linalg.solve

def rayleigh_coeffs(omegas, zetas):

    omegas = np.asarray(omegas, dtype=float)
    zetas  = np.asarray(zetas,  dtype=float)

    ### missing code here ###


    
    return alpha, beta

In [ ]:
omegas=eig_freq[[1,3]]
zetas=[0.002, 0.003]

alpha, beta= rayleigh_coeffs(omegas, zetas)
print(alpha, beta)

In [ ]:
C = alpha*M + beta*K

## FRF generation in Python

We will display the generation of frequency response functions in two different ways. First is the direct harmonic and second is the mode-superposition.

### Direct harmonic

FRFs for the small systems can be efficiently computed using the direct harmonic method:


$$\left(-\omega^2\mathbf{M}+ i\omega \mathbf{C}  + \mathbf{K}\right) \, \boldsymbol{X} = \boldsymbol{F}.$$

So, the FRF matrix can be calculated as follows:

$$\boldsymbol{Y}(\omega) =\frac{\boldsymbol{X}(\omega)}{\boldsymbol{F}(\omega)} = (\mathbf{K}+i\omega \mathbf{C}-\omega^2\mathbf{M})^{-1}.$$

Before generating the FRFs we first have to define the frequency vector:

In [ ]:
f_start = 0.01
f_end = 5
f_resolution = .01
freq = np.arange(f_start, f_end, f_resolution)

ome = 2 * np.pi * freq

In [ ]:
# compute the FRF with the direct method at each frequency line. 

FRF1 = np.zeros((4, 4, len(freq)), dtype=complex)

### missing code here ###


In [ ]:
plt.semilogy(freq,np.abs(FRF1[0,1,:]));
plt.ylabel(r'$|Y|$ [m/N]')
plt.xlabel('f [Hz]')


<div class="alert alert-block alert-warning">
    <i class="fa fa-question-circle" aria-hidden="true"></i>
    <b>Question:</b> How is this FRF analysis numerically performed in a commercial FE software, and why?
</div>

### Mode superposition

For FRF generation we can also use the modal superposition method, where contributions of modes are superposed at each frequency line using the equation: 

$$
Y_{i,j}(\omega)
\;=\;\sum_{r=1}^{m}
\frac{\phi_{i,r}\,\phi_{j,r}}
     {\omega_r^2 \;-\;\omega^2 
      \;+\;i\,\omega\bigl(\alpha + \beta\,\omega_r^2\bigr)}\,.
$$


In [ ]:
# compute the FRF with the mode superposition method at each frequency line. 

FRF2 = np.zeros((4, 4, len(freq)), dtype=complex)

### missing code here ###
    

In [ ]:
plt.semilogy(freq,np.abs(FRF2[0,1,:]));
plt.ylabel(r'$|Y|$ [m/N]')
plt.xlabel('f [Hz]')

Now we can compare the two FRF synthetizations methods:

In [ ]:
plt.semilogy(freq,np.abs(FRF1[0,1,:]), label='direct')
plt.semilogy(freq,np.abs(FRF2[0,1,:]),'--', label='mode superposition');
plt.ylabel(r'$|Y|$ [m/N]')
plt.xlabel('f [Hz]')
plt.legend()

<div class="alert alert-block alert-warning">
    <i class="fa fa-question-circle" aria-hidden="true"></i>
    <b>Question:</b> What is the main issue of mode-superposition for a large FE case? and how can it be corrected? Is there any other alternative to the classic mode-superposition?
</div>

## Recap of frequency based substructuring (FBS) 

Let's now assume two interacting subsystems to be assembled:

<p>
<center><img src="./pics/assembly.png" width=400/></center>
</p>

Combining the equation of motion with the interface conditions, i.e. compatibility of displacements and equilibrium of forces, the frequency-based formulation of the substructuring problem becomes:

$
\begin{cases}
\mathbf{Z}(\omega)\,\boldsymbol{u}(\omega)=\boldsymbol{f}(\omega)+\boldsymbol{g}(\omega) \\ \mathbf{B}\,\boldsymbol{u}(\omega)=\mathbf{0} \\ \mathbf{L}^{\mathrm T}\boldsymbol{g}(\omega)=\mathbf{0}
\end{cases}
$

where $
\mathbf{Z}(\omega)
=
\begin{bmatrix}
\mathbf{Z}^A(\omega) & \mathbf{0} \\[6pt]
\mathbf{0}           & \mathbf{Z}^B(\omega)
\end{bmatrix},
\quad
\boldsymbol{u}(\omega)
=
\begin{bmatrix}
\boldsymbol{u}^A(\omega) \\[4pt]
\boldsymbol{u}^B(\omega)
\end{bmatrix},
\quad
\boldsymbol{f}(\omega)
=
\begin{bmatrix}
\boldsymbol{f}^A(\omega) \\[4pt]
\boldsymbol{f}^B(\omega)
\end{bmatrix},
\quad
\boldsymbol{g}(\omega)
=
\begin{bmatrix}
\boldsymbol{g}^A(\omega) \\[4pt]
\boldsymbol{g}^B(\omega)
\end{bmatrix}.
$


The signed Boolean matrix $\mathbf{B}=
\begin{bmatrix}
\mathbf{0} & -\mathbf{I} &  \mathbf{I} & \mathbf{0} 
\end{bmatrix}$ maps the corresponding matching interface DoFs. Each row of the matrix identifies a single pair of interface DoFs to be connected.


The localization Boolean matrix $\mathbf{L}=
\begin{bmatrix}
\mathbf{I} & \mathbf{0} &  \mathbf{0} \\ 
\mathbf{0} & \mathbf{I} & \mathbf{0} \\
\mathbf{0} & \mathbf{I} &  \mathbf{0} \\
\mathbf{0} & \mathbf{0} &  \mathbf{I} 
\end{bmatrix}
$ maps the physical DoFs of all subsystems to the unique set of interface DoFs such that $\mathbf{L}=\text{null}(\mathbf{B})$.

From here on the frequency-dependence will be omitted for simplicity.

### Dual Formulation: LM-FBS

Starting from the general formulation of the substructuring problem, the dual approach chooses Lagrange multipliers $\boldsymbol{\lambda}$ as set of coupling forces according to the relation $\boldsymbol{g}=-\mathbf{B}^{\mathrm T}\boldsymbol{\lambda}$. The equilibrium is thus satisfied a priori.

The equations of motion of the substructuring problem become:

$\begin{cases}
\mathbf{Z}^\mathrm{A|B}\boldsymbol{u}=\boldsymbol{f}-\mathbf{B}^\mathrm{T}\boldsymbol{\lambda} \\ \mathbf{B}\,\boldsymbol{u}=\mathbf{0} 
\end{cases}$.

To solve the system equations, let's first write them in the admittance notation:

$\begin{cases}
\boldsymbol{u}=\mathbf{Y}^\mathrm{A|B}(\boldsymbol{f}-\mathbf{B}^\mathrm{T}\boldsymbol{\lambda}) \\ \mathbf{B}\,\boldsymbol{u}=\mathbf{0} 
\end{cases}, \qquad \mathbf{Y}^\mathrm{A|B}=\begin{bmatrix}
\mathbf{Y}^\mathrm A & \mathbf{0} \\ 
\mathbf{0} & \mathbf{Y}^\mathrm B
\end{bmatrix}$

By substituting the first line in the second line (compatibility constraint) and solving for $\boldsymbol{\lambda}$:

$\boldsymbol{\lambda}=\left(\mathbf{B}\mathbf{Y}^\mathrm{A|B}\mathbf{B}^\mathrm{T}\right)^{-1}\mathbf{B}\mathbf{Y}^\mathrm{A|B}\boldsymbol{f}$

By substituting back the $\boldsymbol{\lambda}$ in the first line of the governing equation of motion:

$\boldsymbol{u}=(\mathbf{Y}^\mathrm{A|B}-\mathbf{Y}^\mathrm{A|B}\mathbf{B}^\mathrm{T}\left(\mathbf{B}\mathbf{Y}^\mathrm{A|B}\mathbf{B}^\mathrm{T}\right)^{-1}\mathbf{B}\mathbf{Y}^\mathrm{A|B})\boldsymbol{f}$

<div class="alert alert-block alert-warning">
    <i class="fa fa-question-circle" aria-hidden="true"></i>
    <b>Question:</b> Why FBS has become popular among substructuring formulations? 
</div>

## LM-FBS coupling in Python

Two substructures to be coupled are both simple discrete mass systems and each has 4 degrees of freedom. Substructure **A** is depicted in <font color='blue'>**blue**</font>: 

<p>
<img src="./pics/A.svg"/ width = 400>
</p>

And substructure **B** is depicted in <font color='red'>**red**</font> color:


<p>
<img src="./pics/B.svg"/ width = 400>
</p>

The same approach as before is used to synthesize FRFs for substructure **B** that consist of the following masses, springs and rayleigh parameters:

$m_5,\,m_6,\,m_7,\,m_8 = 2\,\text{kg}$

$k_4,\,k_5,\,k_6 = 100\,\text{N/m}$

$\alpha=0.019790752586283235,\ \beta=0.00020062717328036815$

Now let's examine the coupling configuration **AB**. DoFs <font color='blue'>**2, 3, 4**</font> and <font color='red'>**5, 6, 7**</font> from individual substructures are collocated so the assembly looks like this:

<p>
<img src="./pics/A_B.svg"/ width = 400>
</p>

Hence interface between substructures consists of 3 degrees of freedom. The following sets of responses, applied forces, interface forces and uncoupled admittance can be defined from the figure above:

$$\boldsymbol{u} = \begin{bmatrix}u_1^\text{A}\\u_2^\text{A}\\u_3^\text{A}\\u_4^\text{A}\\
                                u_5^\text{B}\\u_6^\text{B}\\u_7^\text{B}\\u_8^\text{B}\end{bmatrix},\quad\boldsymbol{f} = \begin{bmatrix}f_1^\text{A}\\f_2^\text{A}\\f_3^\text{A}\\f_4^\text{A}\\
                                f_5^\text{B}\\f_6^\text{B}\\f_7^\text{B}\\f_8^\text{B}\end{bmatrix},\quad\boldsymbol{g} = \begin{bmatrix}0\\g_2^\text{A}\\g_3^\text{A}\\g_4^\text{A}\\
                                g_5^\text{B}\\g_6^\text{B}\\g_7^\text{B}\\0\end{bmatrix}\quad\mathbf{Y}^{\text{A|B}} = 
                                \begin{bmatrix}
                                Y_{11}^{\text{A}}&Y_{12}^{\text{A}}&Y_{13}^{\text{A}}&Y_{14}^{\text{A}}&0&0&0&0\\
                                Y_{21}^{\text{A}}&Y_{22}^{\text{A}}&Y_{23}^{\text{A}}&Y_{24}^{\text{A}}&0&0&0&0\\
                                Y_{31}^{\text{A}}&Y_{32}^{\text{A}}&Y_{33}^{\text{A}}&Y_{34}^{\text{A}}&0&0&0&0\\
                                Y_{41}^{\text{A}}&Y_{42}^{\text{A}}&Y_{43}^{\text{A}}&Y_{44}^{\text{A}}&0&0&0&0\\
                                0&0&0&0&Y_{55}^{\text{B}}&Y_{56}^{\text{B}}&Y_{57}^{\text{B}}&Y_{58}^{\text{B}}\\
                                0&0&0&0&Y_{65}^{\text{B}}&Y_{66}^{\text{B}}&Y_{67}^{\text{B}}&Y_{68}^{\text{B}}\\
                                0&0&0&0&Y_{75}^{\text{B}}&Y_{76}^{\text{B}}&Y_{77}^{\text{B}}&Y_{78}^{\text{B}}\\
                                0&0&0&0&Y_{85}^{\text{B}}&Y_{86}^{\text{B}}&Y_{87}^{\text{B}}&Y_{88}^{\text{B}}
                                \end{bmatrix}$$

First, we import admittance matrices for each substructure ($\mathbf{Y}^\text{A}$ and $\mathbf{Y}^\text{B}$, respectively). For this example we use a full-DoF response models, which means FRFs for response and excitation at each combination of DoFs must be included.

In [ ]:
Y_A = np.load('./data/Y_A.npy')
Y_B = np.load('./data/Y_B.npy')
Y_A.shape, Y_B.shape

In [ ]:
freq = np.load('./data/freq.npy')
freq.shape

Next, construct an admittance matrix for an uncoupled system, containing substructure admittances:

$$\mathbf{Y}^\text{A|B} = \begin{bmatrix} 
\mathbf{Y}^\text{A} & \mathbf{0} \\
\mathbf{0} & \mathbf{Y}^\text{B}
\end{bmatrix}$$

In [ ]:
# Construct the uncoupled admittance Y_A_B

# prepare matrix full of zeros
Y_A_B = np.zeros((Y_A.shape[0], Y_A.shape[1]+Y_B.shape[1], Y_A.shape[2]+Y_B.shape[2]), dtype=complex) 

### missing code here ###


In [ ]:
# visualize your uncoupled admittance for verification

plt.spy(Y_A_B[0,:,:]) 
plt.xlabel('DoF')
plt.ylabel('DoF')
plt.xticks(np.arange(0,8,1),np.arange(1,9,1))
plt.yticks(np.arange(0,8,1),np.arange(1,9,1));

Now we must define compatibility and equilibrium conditions. Let us start with compatibility and put down all the interface DoFs where displacements of the individual substructures must match:

$$u_2^{\text{A}}=u_5^{\text{B}}\quad\Rightarrow\quad u_5^{\text{B}}-u_2^{\text{A}}=0$$\
$$u_3^{\text{A}}=u_6^{\text{B}}\quad\Rightarrow\quad u_6^{\text{B}}-u_3^{\text{A}}=0$$\
$$u_4^{\text{A}}=u_7^{\text{B}}\quad\Rightarrow\quad u_7^{\text{B}}-u_4^{\text{A}}=0$$

We can now simply form Boolean matrix $\mathbf{B}$ so when multiplying $\mathbf{B}\,\boldsymbol{u}$ we obtain compatibility conditions stated above. We need to obtain three equations, so the number of rows equals 3, and we need to incorporate all 8 DoFs, so 8 is the number is columns. To summarize, shape of $\mathbf{B}$ equals $3\,\times\,8$.

In [ ]:
B = np.zeros((3,8))

Multiplying first row of $\mathbf{B}$ with $\boldsymbol{u}$ should yield $u_5^{\text{B}}-u_2^{\text{A}}=0$:

$$\begin{bmatrix}0&-1&0&0&1&0&0&0\end{bmatrix}\begin{bmatrix}u_1^\text{A}\\u_2^\text{A}\\u_3^\text{A}\\u_4^\text{A}\\
                                u_5^\text{B}\\u_6^\text{B}\\u_7^\text{B}\\u_8^\text{B}\end{bmatrix} = u_5^{\text{B}}-u_2^{\text{A}} = 0$$

We therefore write the first row of $\mathbf{B}$ as:

In [ ]:
B[0,:] = np.array([0,-1,0,0,1,0,0,0])

Then we do the same with the second and third row:

In [ ]:
# Build second and third row of B

### missing code here ###


In [ ]:
# visualize your Boolean matrix for verification

plt.imshow(B)
plt.colorbar(shrink=0.5)
plt.xlabel('DoF')
plt.ylabel('Number of \n compatibility \n conditions')
plt.xticks(np.arange(0,8,1), np.arange(1,9,1))
plt.yticks(np.arange(0,3,1), np.arange(1,4,1));

We can finally apply the LM-FBS coupling equation:

$$\mathbf Y^{\text{AB}} = \mathbf Y^{\text{A|B}} - \mathbf Y^{\text{A|B}}\,\mathbf B^\mathrm{T} \left( \mathbf B \mathbf Y^{\mathrm{A|B}} \mathbf{B}^\mathrm{T} \right)^{-1} \mathbf B \mathbf Y^\text{A|B}$$

In [ ]:
Y_AB = Y_A_B - Y_A_B @ B.T @ np.linalg.inv(B@Y_A_B@B.T) @ B @ Y_A_B
Y_AB.shape

In this manner we obtain the admittance for the following structure:


<p>
<img src="./pics/AB.svg"/ width = 400>
</p>

Let's remove the redundant interface entries from the $\mathbf{Y}^{\text{AB}}$ matrix manually:

In [ ]:
Y_AB_unique = Y_AB[:,[[0],[1],[2],[3],[-1]],[0,1,2,3,-1]]
Y_AB_unique.shape

And for the final step, let us compare coupled FRFs of AB assembly with actual ones:

In [ ]:
Y_AB_true = np.load('./data/Y_AB.npy')

In [ ]:
i = 0
o = 0

plt.subplots(figsize=(10,6))
plt.subplot(211)
plt.semilogy(freq, np.abs(Y_AB_unique[:,o,i]), color='k', label='Coupled FRF')
plt.semilogy(freq, np.abs(Y_AB_true[:,o,i]), '--', color='orange', label='Reference FRF')
plt.legend(loc=0)
plt.ylabel(r'$|Y|$ [m/N]')

plt.subplot(212)
plt.plot(freq, np.angle(Y_AB_unique[:,o,i]), color='k', label='Coupled FRF')
plt.plot(freq, np.angle(Y_AB_true[:,o,i]), '--', color='orange', label='Reference FRF')
plt.xlabel('f [Hz]')
plt.ylabel(r'$\angle Y$ [rad]');

## Takeaway
Identification of interface very difficult for real systems

<div class="alert alert-block alert-warning">
    <i class="fa fa-question-circle" aria-hidden="true"></i>
    <b>Question:</b> The substructuring result is exact on this ideal analytical case. Which issues do you expect to occur in a real experimental scenario?
</div>